In [39]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [40]:
import os 
os.getcwd()

'e:\\딥러닝파이토치교과서\\chap_02'

In [41]:
dataset = pd.read_csv(r"data/car_evaluation.csv")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   price         1728 non-null   object
 1   maint         1728 non-null   object
 2   doors         1728 non-null   object
 3   persons       1728 non-null   object
 4   lug_capacity  1728 non-null   object
 5   safety        1728 non-null   object
 6   output        1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [42]:
dataset.head()

price  maint doors persons lug_capacity safety output
0  vhigh  vhigh     2       2        small    low  unacc
1  vhigh  vhigh     2       2        small    med  unacc
2  vhigh  vhigh     2       2        small   high  unacc
3  vhigh  vhigh     2       2          med    low  unacc
4  vhigh  vhigh     2       2          med    med  unacc

In [43]:
dataset["output"].value_counts()

unacc    1210
acc       384
good       69
vgood      65
Name: output, dtype: int64

In [44]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0], fig_size[1] = 10, 6
plt.rcParams["figure.figsize"] = fig_size
dataset["output"].value_counts().plot(kind='pie', autopct="%0.05f%%", colors=['lightblue','lightgreen','orange','pink'], explode=(0.05, 0.05, 0.05, 0.05))
plt.show()

In [45]:
categorical_columns = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety']

In [46]:
for category in categorical_columns:
    dataset[category] = dataset[category].astype('category')

In [47]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   price         1728 non-null   category
 1   maint         1728 non-null   category
 2   doors         1728 non-null   category
 3   persons       1728 non-null   category
 4   lug_capacity  1728 non-null   category
 5   safety        1728 non-null   category
 6   output        1728 non-null   object  
dtypes: category(6), object(1)
memory usage: 24.7+ KB


In [48]:
price = dataset['price'].cat.codes.values
maint = dataset['maint'].cat.codes.values
doors = dataset['doors'].cat.codes.values
persons = dataset['persons'].cat.codes.values
lug_capacity = dataset['lug_capacity'].cat.codes.values
safety = dataset['safety'].cat.codes.values

categorical_data = np.stack([price, maint, doors, persons, lug_capacity, safety], axis=1)
print(categorical_data.shape)
print(categorical_data[:10])

(1728, 6)
[[3 3 0 0 2 1]
 [3 3 0 0 2 2]
 [3 3 0 0 2 0]
 [3 3 0 0 1 1]
 [3 3 0 0 1 2]
 [3 3 0 0 1 0]
 [3 3 0 0 0 1]
 [3 3 0 0 0 2]
 [3 3 0 0 0 0]
 [3 3 0 1 2 1]]


In [49]:
categorical_data = torch.tensor(categorical_data, dtype=torch.int64)
categorical_data[:10]

tensor([[3, 3, 0, 0, 2, 1],
        [3, 3, 0, 0, 2, 2],
        [3, 3, 0, 0, 2, 0],
        [3, 3, 0, 0, 1, 1],
        [3, 3, 0, 0, 1, 2],
        [3, 3, 0, 0, 1, 0],
        [3, 3, 0, 0, 0, 1],
        [3, 3, 0, 0, 0, 2],
        [3, 3, 0, 0, 0, 0],
        [3, 3, 0, 1, 2, 1]])

In [50]:
outputs = pd.get_dummies(dataset['output'])
print(outputs)

      acc  good  unacc  vgood
0       0     0      1      0
1       0     0      1      0
2       0     0      1      0
3       0     0      1      0
4       0     0      1      0
...   ...   ...    ...    ...
1723    0     1      0      0
1724    0     0      0      1
1725    0     0      1      0
1726    0     1      0      0
1727    0     0      0      1

[1728 rows x 4 columns]


In [51]:
outputs = outputs.values
outputs = torch.tensor(outputs)
print(outputs)

tensor([[0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        ...,
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1]], dtype=torch.uint8)


In [52]:
outputs = outputs.flatten()
print(outputs)

tensor([0, 0, 1,  ..., 0, 0, 1], dtype=torch.uint8)


In [53]:
print(categorical_data.shape)
print(outputs.shape)

torch.Size([1728, 6])
torch.Size([6912])


In [54]:
dataset['price'].cat.categories

Index(['high', 'low', 'med', 'vhigh'], dtype='object')

In [55]:
categorical_column_sizes = [len(dataset[column].cat.categories) for column in categorical_columns]
print(categorical_column_sizes)

[4, 4, 4, 3, 3, 3]


In [56]:
categorical_embedding_sizes = [(col_size, min(50, (col_size + 1)//2)) for col_size in categorical_column_sizes]
print(categorical_embedding_sizes)

[(4, 2), (4, 2), (4, 2), (3, 2), (3, 2), (3, 2)]


In [57]:
total_records = 1728
test_records = int(total_records * .2)

categorical_train_data = categorical_data[: total_records - test_records]
categorical_test_data = categorical_data[total_records - test_records : total_records]

train_outputs = outputs[: total_records - test_records]
test_outputs = outputs [total_records - test_records : total_records]

In [58]:
print(f"categorical_train_data의 길이 : {len(categorical_train_data)},  train_outputs의 길이 : {len(train_outputs)}")
print(f"categorical_test_data의 길이  : {len(categorical_test_data)},   test_outputs의 길이  : {len(test_outputs)}")

categorical_train_data의 길이 : 1383,  train_outputs의 길이 : 1383
categorical_test_data의 길이  : 345,   test_outputs의 길이  : 345


In [59]:
class Model(nn.Module):
    def __init__(self, embedding_size, output_size, layers, p=0.4):
        super().__init__()
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(p)
        
        all_layers = []
        num_categorical_cols = sum((nf for ni, nf in embedding_size))
        input_size = num_categorical_cols
        
        for i in layers:
            all_layers.append(nn.Linear(input_size, i))
            all_layers.append(nn.ReLU(inplace=True))
            all_layers.append(nn.BatchNorm1d(i))
            all_layers.append(nn.Dropout(p))
            input_size = i
            
        all_layers.append(nn.Linear(layers[-1], output_size))
        self.layers = nn.Sequential(*all_layers)
        
        
    def forward(self, x_categorical):
        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        x = torch.cat(embeddings, 1)
        x = self.embedding_dropout(x)
        x = self.layers(x)
        return x

In [60]:
model = Model(categorical_embedding_sizes, 4, [200,100,50], p=0.4)
print(model)

Model(
  (all_embeddings): ModuleList(
    (0): Embedding(4, 2)
    (1): Embedding(4, 2)
    (2): Embedding(4, 2)
    (3): Embedding(3, 2)
    (4): Embedding(3, 2)
    (5): Embedding(3, 2)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=12, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=100, out_features=50, bias=True)
    (9): ReLU(inplace=True)
    (10): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.4, inplace=False)
    (12): Linear(in_features=50, out_features=4, bias=True)
  )
)


In [61]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [62]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [73]:
epochs = 500
aggregated_losses = []
train_outputs = train_outputs.to(device=device, dtype=torch.int64)

for i in range(epochs):
    i += 1
    y_pred = model(categorical_train_data).to(device=device)
    single_loss = loss_function(y_pred, train_outputs)
    aggregated_losses.append(single_loss)
    
    if i % 25 == 1:
        print(f"epoch : {i:3}   loss : {single_loss.item():10.8f}")
        
    optimizer.zero_grad()
    single_loss.backward()
    optimizer.step()
    
print(f"epoch : {i:3}  loss : {single_loss.item():10.10f}")

epoch :   1   loss : 1.58716011
epoch :  26   loss : 1.39414096
epoch :  51   loss : 1.29236090
epoch :  76   loss : 1.18253875
epoch : 101   loss : 1.07452559
epoch : 126   loss : 0.94462031
epoch : 151   loss : 0.84085155
epoch : 176   loss : 0.75246233
epoch : 201   loss : 0.69595999
epoch : 226   loss : 0.67035633
epoch : 251   loss : 0.64368510
epoch : 276   loss : 0.61063641
epoch : 301   loss : 0.60099643
epoch : 326   loss : 0.59549803
epoch : 351   loss : 0.60375112
epoch : 376   loss : 0.59186608
epoch : 401   loss : 0.58106768
epoch : 426   loss : 0.57962561
epoch : 451   loss : 0.57815260
epoch : 476   loss : 0.57122940
epoch : 500  loss : 0.5720500350


In [74]:
test_outputs = test_outputs.to(device=device, dtype=torch.int64)
with torch.no_grad():
    y_val = model(categorical_test_data).to(device=device)
    loss = loss_function(y_val, test_outputs)
print(f"Loss : {loss:.8f}")

Loss : 0.54174381


In [75]:
print(y_val[:5])

tensor([[ 3.3186,  2.0832, -3.4375, -3.2950],
        [ 2.0761,  0.9828, -3.6870, -3.3400],
        [ 1.3967,  0.7658, -2.6130, -2.6083],
        [ 2.8678,  1.7297, -2.7888, -2.9110],
        [ 2.8111,  1.5470, -3.1590, -2.8906]], device='cuda:0')


In [77]:
y_val = y_val.to('cpu')
y_val = np.argmax(y_val, axis=1)
print(y_val[:5])

tensor([0, 0, 0, 0, 0])


In [81]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

test_outputs = test_outputs.to('cpu')

In [82]:
print(classification_report(test_outputs, y_val))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86       259
           1       0.67      0.02      0.04        86

    accuracy                           0.75       345
   macro avg       0.71      0.51      0.45       345
weighted avg       0.73      0.75      0.66       345



In [85]:
print(confusion_matrix(test_outputs, y_val))

[[258   1]
 [ 84   2]]


In [86]:
print(accuracy_score(test_outputs, y_val))

0.7536231884057971
